In [1]:
# Dependencies
# ----------------------------------

import os

import numpy as np
import pandas as pd

# Import the method used for connecting to DBs
from sqlalchemy import create_engine

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

# Import the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Import Session to bind to the DB
from sqlalchemy.orm import Session


In [2]:
## OS command to get the cwd and change the cwd
##---------------------------------------------
## Get the current wroking directory
#os.getcwd()
## In the path the backward slash (\) needs to be escaped with another backslash (\)
#os.chdir("C:\\Users\\sanji\\Documents\\project\\p2-earthquake-visualization\\earthquake")
#os.chdir("C:\\Users\\sanji\\Documents\\project\\p2-earthquake-visualization")
## Verify the cwd after the change
#os.getcwd()

In [3]:
# Define the raw data csv file to load, process and later save it to the database
#----------------------------------------------
earthquake_csv = "static/data/earthquake.csv"

# Read the csv data file into pandas dataframe
#------------------------------------------------
earthquake_df = pd.read_csv(earthquake_csv, encoding="ISO-8859-1")

In [4]:
# Show just the header
#------------------------
earthquake_df.head()

,ID,FLAG_TSUNAMI,YEAR,MONTH,DAY,FOCAL_DEPTH,EQ_PRIMARY,INTENSITY,COUNTRY,STATE,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
0,1,1,-2150,NaN,NaN,NaN,7.3,NaN,JORDAN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0,-2000,NaN,NaN,18.0,7.1,10.0,TURKMENISTAN,NaN,...,NaN,NaN,NaN,NaN,NaN,1--limited(~$1 million),NaN,1--Few(~1 to 50 houses),NaN,NaN
2,2,1,-2000,NaN,NaN,NaN,NaN,10.0,SYRIA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5877,1,-1610,NaN,NaN,NaN,NaN,NaN,GREECE,NaN,...,NaN,NaN,NaN,NaN,NaN,3--Severe(~$5 to $24 million),NaN,NaN,NaN,NaN
4,8,0,-1566,NaN,NaN,NaN,NaN,10.0,ISRAEL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Get the columns in the dataFrame
#---------------------------------------------------------------
earthquake_df.columns

Index(['ID', 'FLAG_TSUNAMI', 'YEAR', 'MONTH', 'DAY', 'FOCAL_DEPTH',
       'EQ_PRIMARY', 'INTENSITY', 'COUNTRY', 'STATE', 'LOCATION_NAME',
       'LATITUDE', 'LONGITUDE', 'REGION_CODE', 'REGION', 'TOTAL_DEATHS',
       'TOTAL_DEATHS_DESCRIPTION', 'TOTAL_MISSING',
       'TOTAL_MISSING_DESCRIPTION', 'TOTAL_INJURIES',
       'TOTAL_INJURIES_DESCRIPTION', 'TOTAL_DAMAGE_MILLIONS_DOLLARS',
       'TOTAL_DAMAGE_DESCRIPTION', 'TOTAL_HOUSES_DESTROYED',
       'TOTAL_HOUSES_DESTROYED_DESCRIPTION', 'TOTAL_HOUSES_DAMAGED',
       'TOTAL_HOUSES_DAMAGED_DESCRIPTION'],
      dtype='object')

In [6]:
# Specify the columns that will need to be loaded into the table
#---------------------------------------------------------------
edited_earthquake_df = earthquake_df[['ID','FLAG_TSUNAMI','YEAR','EQ_PRIMARY','INTENSITY','COUNTRY','STATE','LOCATION_NAME','LATITUDE','LONGITUDE','REGION_CODE','REGION','TOTAL_DEATHS_DESCRIPTION','TOTAL_MISSING_DESCRIPTION','TOTAL_INJURIES_DESCRIPTION','TOTAL_DAMAGE_DESCRIPTION','TOTAL_HOUSES_DESTROYED_DESCRIPTION','TOTAL_HOUSES_DAMAGED_DESCRIPTION']]

In [7]:
# Show just the header
#------------------------

edited_earthquake_df.head()

,ID,FLAG_TSUNAMI,YEAR,EQ_PRIMARY,INTENSITY,COUNTRY,STATE,LOCATION_NAME,LATITUDE,LONGITUDE,REGION_CODE,REGION,TOTAL_DEATHS_DESCRIPTION,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED_DESCRIPTION
0,1,1,-2150,7.3,NaN,JORDAN,NaN,"JORDAN: BAB-A-DARAA,AL-KARAK",31.1,35.5,140,Middle East,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0,-2000,7.1,10.0,TURKMENISTAN,NaN,TURKMENISTAN: W,38,58.2,40,Central Asia and Caucasus,1--Few (~1 to 50 deaths),NaN,NaN,1--limited(~$1 million),1--Few(~1 to 50 houses),NaN
2,2,1,-2000,NaN,10.0,SYRIA,NaN,SYRIA: UGARIT,35.683,35.8,130,Southern Europe,3--Many (~101 to 1000 deaths),NaN,NaN,NaN,NaN,NaN
3,5877,1,-1610,NaN,NaN,GREECE,NaN,GREECE: THERA ISLAND (SANTORINI),36.4,25.4,130,Southern Europe,3--Many (~101 to 1000 deaths),NaN,NaN,3--Severe(~$5 to $24 million),NaN,NaN
4,8,0,-1566,NaN,10.0,ISRAEL,NaN,ISRAEL: ARIHA (JERICHO),31.5,35.3,140,Middle East,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Get the columns in the dataFrame
#---------------------------------------------------------------
edited_earthquake_df.columns

Index(['ID', 'FLAG_TSUNAMI', 'YEAR', 'EQ_PRIMARY', 'INTENSITY', 'COUNTRY',
       'STATE', 'LOCATION_NAME', 'LATITUDE', 'LONGITUDE', 'REGION_CODE',
       'REGION', 'TOTAL_DEATHS_DESCRIPTION', 'TOTAL_MISSING_DESCRIPTION',
       'TOTAL_INJURIES_DESCRIPTION', 'TOTAL_DAMAGE_DESCRIPTION',
       'TOTAL_HOUSES_DESTROYED_DESCRIPTION',
       'TOTAL_HOUSES_DAMAGED_DESCRIPTION'],
      dtype='object')

In [9]:
# Rename the DataFrame columns
#---------------------------------
edited2_earthquake_df = edited_earthquake_df.rename(columns={
    'ID':'id',
    'FLAG_TSUNAMI': 'tsunami_fl', 
    'YEAR':'year', 
    'EQ_PRIMARY':'magnitude',
    'INTENSITY':'intensity',
    'COUNTRY':'country',
    'STATE':'state',
    'LOCATION_NAME':'location',
    'LATITUDE':'lat',
    'LONGITUDE':'lng',
    'REGION_CODE':'region_cd',
    'REGION':'region',
    'TOTAL_DEATHS_DESCRIPTION':'total_deaths_desc',
    'TOTAL_MISSING_DESCRIPTION':'total_missing_desc',
    'TOTAL_INJURIES_DESCRIPTION':'total_injuries_desc',
    'TOTAL_DAMAGE_DESCRIPTION':'total_damage_desc',
    'TOTAL_HOUSES_DESTROYED_DESCRIPTION':'total_houses_destroyed_desc',
    'TOTAL_HOUSES_DAMAGED_DESCRIPTION':'total_houses_damaged_desc'
})

In [10]:
edited2_earthquake_df.head()

,id,tsunami_fl,year,magnitude,intensity,country,state,location,lat,lng,region_cd,region,total_deaths_desc,total_missing_desc,total_injuries_desc,total_damage_desc,total_houses_destroyed_desc,total_houses_damaged_desc
0,1,1,-2150,7.3,NaN,JORDAN,NaN,"JORDAN: BAB-A-DARAA,AL-KARAK",31.1,35.5,140,Middle East,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0,-2000,7.1,10.0,TURKMENISTAN,NaN,TURKMENISTAN: W,38,58.2,40,Central Asia and Caucasus,1--Few (~1 to 50 deaths),NaN,NaN,1--limited(~$1 million),1--Few(~1 to 50 houses),NaN
2,2,1,-2000,NaN,10.0,SYRIA,NaN,SYRIA: UGARIT,35.683,35.8,130,Southern Europe,3--Many (~101 to 1000 deaths),NaN,NaN,NaN,NaN,NaN
3,5877,1,-1610,NaN,NaN,GREECE,NaN,GREECE: THERA ISLAND (SANTORINI),36.4,25.4,130,Southern Europe,3--Many (~101 to 1000 deaths),NaN,NaN,3--Severe(~$5 to $24 million),NaN,NaN
4,8,0,-1566,NaN,10.0,ISRAEL,NaN,ISRAEL: ARIHA (JERICHO),31.5,35.3,140,Middle East,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Create Class - Earthquake
#---------------------------------------------
# Set an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Creates Class which will serve as the anchor point for the Table
class Earthquake(Base):
    __tablename__ = 'earthquake'
    id = Column(Integer, primary_key=True)
    tsunami_fl = Column(Integer, nullable=False)
    year = Column(String(10))
    magnitude = Column(Integer)
    intensity = Column(Float)
    country = Column(String(55), nullable=False)
    state = Column(String(55))
    location = Column(String(55))
    lat = Column(Float)
    lng = Column(Float)
    region_cd = Column(String(55), nullable=False)
    region = Column(String(55), nullable=False)
    total_deaths_desc = Column(String(255))
    total_missing_desc = Column(String(255))
    total_injuries_desc = Column(String(255))
    total_damage_desc = Column(String(255))
    total_houses_destroyed_desc = Column(String(255))
    total_houses_damaged_desc = Column(String(255))    

In [17]:
# Create Database Connection
# ----------------------------------

conn_string = "sqlite:///static/data/earthquake.sqlite"
engine = create_engine(conn_string, echo=True)
conn = engine.connect()


2020-06-27 10:44:09,268 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-06-27 10:44:09,270 INFO sqlalchemy.engine.base.Engine ()
2020-06-27 10:44:09,272 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-06-27 10:44:09,274 INFO sqlalchemy.engine.base.Engine ()


In [18]:
# Create a "Metadata" Layer that Abstracts the SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

2020-06-27 10:44:10,990 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("earthquake")
2020-06-27 10:44:10,991 INFO sqlalchemy.engine.base.Engine ()


In [19]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
session = Session(bind=engine)

In [20]:
edited2_earthquake_df.to_sql('earthquake', conn, if_exists='replace', index = False)

2020-06-27 10:44:12,927 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("earthquake")
2020-06-27 10:44:12,928 INFO sqlalchemy.engine.base.Engine ()
2020-06-27 10:44:12,930 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("earthquake")
2020-06-27 10:44:12,931 INFO sqlalchemy.engine.base.Engine ()
2020-06-27 10:44:12,933 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-06-27 10:44:12,935 INFO sqlalchemy.engine.base.Engine ()
2020-06-27 10:44:12,937 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("earthquake")
2020-06-27 10:44:12,939 INFO sqlalchemy.engine.base.Engine ()
2020-06-27 10:44:12,943 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'earthquake' AND type = 'table'
2020-06-27 10:44:12,944 INFO sqlalchemy.engine.base.Engine ()
2020-06-27 10:44:12,946 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_

In [21]:
session.commit()

In [22]:
#cur = conn.cursor()
#cur.execute("select * from earthquake limit 5;")

In [24]:
with conn as con:
    
    rs = con.execute("""SELECT * FROM earthquake limit 5;""")
    
    for row in rs:
        print(row)
        
#con.close()

StatementError: (sqlalchemy.exc.ResourceClosedError) This Connection is closed
[SQL: SELECT * FROM earthquake limit 5;]

In [25]:
con.close()